In [141]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [142]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).text

### Parse wiki doc

In [143]:
soup = BeautifulSoup(html,'html.parser')

In [144]:
df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
rows = soup.findAll('tr')
i=-0
for tr in rows:
    cells =tr.findAll('td')
    c1= [c.text for c in cells]
    if(len(c1) != 0 and c1[0].startswith('M') and ~c1[1].find("Not assigned")>=0):    
        df.loc[i] = [c1[0],c1[1],c1[2] ]
        i=i+1
       

In [145]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [146]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [147]:
df.shape

(212, 3)

#### Replace unwanted text '\n' at the end of Neighborhood

In [148]:
df['Neighborhood'] = df['Neighborhood'].str.replace('\n','')

In [149]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### If borough is there but Neighborhood is not assigned then borough and neighborhood should be same

In [150]:
#Borough is not assigned
df[df['Borough'].str.contains('Park')]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [151]:
#check
df.loc[df['Neighborhood']=='Not assigned','PostalCode']

6    M7A
Name: PostalCode, dtype: object

In [152]:
#Assign all neighborhoods to borough whose neighborhood are not assigned
df.loc[df['Neighborhood']=='Not assigned','Neighborhood'] = df.loc[df['Neighborhood']=='Not assigned','Borough'] 


In [153]:
#Checck after assignment
df[df['Borough'].str.contains('Park')]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Queen's Park


#### Merger more than one same neighborhood into a single row sepearted by columns

In [154]:
# Example row
df[df['PostalCode'].str.contains('M5A')]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park


In [155]:
#Merge all rows and create dataframe
dfNew = df.groupby(['PostalCode','Borough'],sort=False).agg(lambda col: ', '.join(col))
dfNew.reset_index(inplace=True)
dfNew[dfNew['PostalCode'].str.contains('M5A')]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [158]:
df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [156]:
dfNew.shape

(103, 3)